In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [7]:
from pyspark.sql.types import DoubleType

In [8]:
df = sqlContext.read.csv('hdfs://master:9000/user/hadoop/land_value_adm')
df = df.withColumn('_c1', df['_c1'].cast(DoubleType()))
df.show()

+-------------------+---------+----------+
|                _c0|      _c1|       _c2|
+-------------------+---------+----------+
|1111010200100010028|2084400.0|1111051500|
|1111010500100210013|4045600.0|1111051500|
|1111010600100250004|3296200.0|1111053000|
|1111010800101470001|7340800.0|1111051500|
|1111010900100230000|2660000.0|1111051500|
|1111010900100380001|3249200.0|1111051500|
|1111010900101660077|2928000.0|1111051500|
|1111011000100780003|4866000.0|1111051500|
|1111011100100190012|3014800.0|1111051500|
|1111011100100470065|3179600.0|1111051500|
|1111011100100470162|2553000.0|1111051500|
|1111011100101100000|2991000.0|1111051500|
|1111011200101190000|3204200.0|1111053000|
|1111012000102390000|  4.017E7|1111053000|
|1111012100100010165|3542400.0|1111053000|
|1111012100101080007|8732400.0|1111053000|
|1111012200100430002|  1.886E7|1111061500|
|1111012500101100002| 1.2274E7|1111061500|
|1111012600100420000|   3.63E7|1111061500|
|1111012600100440000|   3.63E7|1111061500|
+----------

In [10]:
df_avg = df.groupBy('_c2').mean('_c1')
df_avg = df_avg.withColumn('_c1', df_avg['avg(_c1)'])[['_c2', '_c1']]
df_avg.show()

+----------+------------------+
|       _c2|               _c1|
+----------+------------------+
|1111054000| 4868209.412191582|
|1153079000|2063868.9651665392|
|1156065000|3407849.5128939827|
|1135061200|2334133.0434782607|
|1123056000| 3081213.012337591|
|1138053000|2500079.8490901017|
|1144055500| 5637254.600606673|
|1147051000| 4981972.296450939|
|1135072000|2965031.1475409837|
|1144060000| 4628329.876054852|
|1144071000|  3181079.27140255|
|1126057500|2353321.0019646366|
|1126068000|2128062.2818086226|
|1132052200|2312008.5566273667|
|1138063200|2188598.7352380953|
|1162072500| 3526439.861297539|
|1168064000| 9545820.250421382|
|1174052000|3773701.6768916156|
|1120056000|3579977.6033755275|
|1162062500|2946740.9531866894|
+----------+------------------+
only showing top 20 rows



In [11]:
maxValue = df_avg.agg({'_c1': 'max'}).collect()[0][0]
maxValue

21532591.55172414

In [12]:
df_norm = df_avg.withColumn('norm', df_avg['_c1'] / maxValue)
df_norm.show()

+----------+------------------+-------------------+
|       _c2|               _c1|               norm|
+----------+------------------+-------------------+
|1111054000| 4868209.412191582|0.22608562469118024|
|1153079000|2063868.9651665392|0.09584860977875571|
|1156065000|3407849.5128939827|0.15826471721752008|
|1135061200|2334133.0434782607|0.10840000553910864|
|1123056000| 3081213.012337591|0.14309531692625613|
|1138053000|2500079.8490901017|  0.116106779023072|
|1144055500| 5637254.600606673|0.26180102785422926|
|1147051000| 4981972.296450939|0.23136891277037328|
|1135072000|2965031.1475409837| 0.1376996884196956|
|1144060000| 4628329.876054852| 0.2149453243905635|
|1144071000|  3181079.27140255|0.14773322866228972|
|1126057500|2353321.0019646366|0.10929111789965679|
|1126068000|2128062.2818086226| 0.0988298262518348|
|1132052200|2312008.5566273667|0.10737251719439417|
|1138063200|2188598.7352380953|0.10164121350561965|
|1162072500| 3526439.861297539| 0.1637721986611116|
|1168064000|

In [14]:
df_norm[df_norm['_c2'] == '1130559500'].show()

+----------+------------------+------------------+
|       _c2|               _c1|              norm|
+----------+------------------+------------------+
|1130559500|2275033.4176479136|0.1056553463238729|
+----------+------------------+------------------+



In [13]:
df_norm.repartition(1).write.csv('hdfs://master:9000/user/hadoop/land_value_burn')